In [50]:
!pip install plotly

    100% |████████████████████████████████| 41.2MB 901kB/s eta 0:00:01    51% |████████████████▌               | 21.3MB 7.9MB/s eta 0:00:03    53% |█████████████████               | 22.0MB 18.5MB/s eta 0:00:02    92% |█████████████████████████████▋  | 38.2MB 11.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
pyspark 2.3.0 requires py4j==0.10.6, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [154]:
!pip install wikipedia

  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
pyspark 2.3.0 requires py4j==0.10.6, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [156]:
import pandas as pd
import requests

In [1]:
import wikipedia as wp

## Load the data and simple cleaning

In [158]:
#Get the html source
html = wp.page("List of states and territories of the United States by population").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"Rank in the fifty states, 2018",Rank in states & territories,Name,"Population estimate, July 1, 2018[5]","Census population, April 1, 2010[6]",Percent increase from 2010-2018[note 1],Total seats in the U.S. House of Representativ...,"Estimated population per electoral vote, 2018[...","Estimated population per House seat, 2018","Census population per House seat, 2010","Percent of the total U.S. population, 2018[not..."
1,1,1,California,39557045,37254523,6.2%,53,719219,746359,702885,11.96%
2,2,2,Texas,28701845,25145561,14.1%,36,755312,797273,698503,8.68%
3,3,3,Florida,21299325,18801310,13.3%,27,734459,788864,696468,6.44%
4,4,4,New York,19542209,19378102,0.8%,27,673869,723786,717707,5.91%


In [177]:
dat = df
#dat.state_name = dat.state_name.str.strip()
dat.columns = dat.iloc[0].values.flatten().tolist()
dat = dat.drop([0], axis=0)
dat.rename(columns = {'Name': 'state_name', 'Population estimate, July 1, 2018[5]' : 'population_2018', 
                     'Census population, April 1, 2010[6]' : 'population_2010', 
                     'Percent increase from 2010-2018[note 1]': 'percent_increase'}, inplace=True)
dat.population_2018 = dat.population_2018.apply(lambda x : int(x.replace(',','').split('[')[0]))
dat.population_2010 = dat.population_2010.apply(lambda x : int(x.replace(',','').split('[')[0]))
dat.percent_increase = dat.percent_increase.apply(lambda x : float(x.replace('%','')))

In [178]:
codes = pd.read_csv('state_codes.csv')
dat = dat.merge(codes, how='left')

In [174]:
dat.dtypes

Rank in the fifty states, 2018                                   object
Rank in states & territories                                     object
state_name                                                       object
population_2018                                                   int64
population_2010                                                   int64
percent_increase                                                 object
Total seats in the U.S. House of Representatives, 2013â2023    object
Estimated population per electoral vote, 2018[note 2]            object
Estimated population per House seat, 2018                        object
Census population per House seat, 2010                           object
Percent of the total U.S. population, 2018[note 3]               object
dtype: object

In [188]:
dat = dat.loc[dat.state_code.isnull() == False]


## Visualization 

In [180]:
import plotly.offline as py
py.init_notebook_mode(connected=True)

This is a US map with growth per state colored

In [208]:
dat_grow = dat.loc[dat.percent_increase >-5]
#dat_grow = dat
import plotly.graph_objs as go

scl = [
    [0.0, 'rgb(242,240,247)'],
    [0.2, 'rgb(218,218,235)'],
    [0.4, 'rgb(188,189,220)'],
    [0.6, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = dat_grow['state_code'],
    z = dat_grow.percent_increase.astype(float),
#    z = dat_grow.population_2018 - dat_grow.population_2010,
    locationmode = 'USA-states',
    text = dat_grow['state_name'],
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 2
        )),
    colorbar = go.choropleth.ColorBar(
        title = "% increase")
#        title = "Numerical increase")
)]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Population Change'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
)

fig3 = go.Figure(data = data, layout = layout)
py.plot(fig3, filename = 'd3-cloropleth-map.html')

'd3-cloropleth-map.html'

This is a per state breakdown of 2010 and 2018 data

In [206]:
import math
fig = {
    'data': [
  		{
  			'x': dat.state_name, 
        	'y': dat.population_2018, 
        	'text': dat.state_name, 
        	'mode': 'markers',
            'marker': {
                'color' : 'green'
            },
        	'name': 'population2018'},
  			{'x': dat.state_name, 
        	'y': dat.population_2010, 
        	'text': dat.state_name, 
        	'mode': 'markers', 
            'marker': {
                'color' : 'red'
            },
        	'name': 'population2010'}        
    ],
    'layout': {
        'xaxis': {'title': 'State name'},
        'yaxis': {'title': "population", 'range': [0, 4.2e7]}
    }
}

# IPython notebook
# py.iplot(fig, filename='pandas/multiple-scatter')

url = py.plot(fig, filename = 'population.html')